In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 
                'occupation', 'relationship', 'race','sex','capital_gain','capital_loss',
                'hours_per_week','native_country','income']

adult_train_df = pd.read_csv('./uci_repos/adults/adult.data',names=column_names, na_values=['?'])
adult_test_df = pd.read_csv('./uci_repos/adults/adult.test', names=column_names,skiprows=[0], na_values=['?']) 

In [3]:
len(adult_train_df), len(adult_test_df)

(32561, 16281)

# Preprocessing

In [4]:
adult_test_df = adult_test_df.drop(['education','age'], axis=1)
adult_train_df = adult_train_df.drop(['education', 'age'], axis=1)

adult_test_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,?,103497,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,Private,215419,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,?,321403,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,Private,374983,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,Private,83891,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [5]:
scaler = StandardScaler()
scale_columns = ['fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']
adult_test_df[scale_columns] = scaler.fit_transform(adult_test_df[scale_columns])
adult_train_df[scale_columns] = scaler.fit_transform(adult_train_df[scale_columns])

for feature in ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']:
    le = LabelEncoder()
    adult_train_df[feature] = le.fit_transform(adult_train_df[feature])
    adult_test_df[feature] = le.fit_transform(adult_test_df[feature])

In [6]:
adult_train_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,7,-1.063611,1.134739,4,1,1,4,1,0.148453,-0.21666,-0.035429,39,0
1,6,-1.008707,1.134739,2,4,0,4,1,-0.145920,-0.21666,-2.222153,39,0
2,4,0.245079,-0.420060,0,6,1,4,1,-0.145920,-0.21666,-0.035429,39,0
3,4,0.425801,-1.197459,2,6,0,2,1,-0.145920,-0.21666,-0.035429,39,0
4,4,1.408176,1.134739,2,10,5,2,0,-0.145920,-0.21666,-0.035429,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,4,0.639741,0.746039,2,13,5,4,0,-0.145920,-0.21666,-0.197409,39,0
32557,4,-0.335433,-0.420060,2,7,0,4,1,-0.145920,-0.21666,-0.035429,39,1
32558,4,-0.358777,-0.420060,6,1,4,4,0,-0.145920,-0.21666,-0.035429,39,0
32559,4,0.110960,-0.420060,4,1,3,4,1,-0.145920,-0.21666,-1.655225,39,0


In [7]:
adult_test_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,4,0.353474,-1.196864,4,7,3,2,1,-0.142662,-0.218062,-0.031432,38,0
1,4,-0.942391,-0.417886,2,5,0,4,1,-0.142662,-0.218062,0.769918,38,0
2,2,1.395450,0.750582,2,11,0,4,1,-0.142662,-0.218062,-0.031432,38,1
3,4,-0.275397,-0.028397,2,7,0,2,1,0.871091,-0.218062,-0.031432,38,1
4,0,-0.812954,-0.028397,4,0,3,4,0,-0.142662,-0.218062,-0.832781,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,4,0.245794,1.140071,0,10,1,4,0,-0.142662,-0.218062,-0.351972,38,0
16277,0,1.248371,-0.417886,6,0,2,2,1,-0.142662,-0.218062,-0.031432,38,0
16278,4,1.755221,1.140071,2,10,0,4,1,-0.142662,-0.218062,0.769918,38,0
16279,4,-0.998420,1.140071,0,1,3,1,1,0.576644,-0.218062,-0.031432,38,0


In [8]:
X_train = adult_train_df.drop(columns=['income']).to_numpy()
y_train = (adult_train_df.income).to_numpy()
X_test= adult_test_df.drop(columns=['income']).to_numpy()
y_test = (adult_test_df.income).to_numpy()

In [9]:
'train data :', X_train.shape, 'test data :', X_test.shape

('train data :', (32561, 12), 'test data :', (16281, 12))

In [10]:
y_test

array([0, 0, 1, ..., 0, 0, 1])

In [11]:
len(adult_test_df.native_country.unique())

41

# VAE Model 

In [12]:
import tensorflow.compat.v1  as tf
tf.disable_v2_behavior()
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import numpy as np

mb_size = 32
z_dim = 3
X_dim = 12
y_dim = 2
h_dim = 3
lr = 1e-3

    
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random.normal(shape=size, stddev=xavier_stddev)

X = tf.keras.Input(shape=(X_dim,))
c = tf.keras.Input(shape=(y_dim,))
z = tf.keras.Input(shape=(z_dim,))

Q_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X, c):
    inputs = tf.concat(axis=1, values=[X, c])
    h = tf.nn.relu(tf.matmul(inputs, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random.normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps

P_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    h = tf.nn.relu(tf.matmul(inputs, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits

z_mu, z_logvar = Q(X, c)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample, c)

X_samples, _ = P(z, c)

recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
vae_loss = tf.reduce_mean(recon_loss + kl_loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
def generate_sample():
    samples = []
    gen_labels =[]
    for r in range(100):
        for index in range(y_dim):
            gen_labels = gen_labels + [1]*mb_size #1 for >50
            y = np.zeros([mb_size, y_dim])
            y[range(mb_size), 1] = 1 #1 for >50 class only
            samples.extend(sess.run(X_samples,
                                   feed_dict={z: np.random.randn(mb_size, z_dim), c: y}))

    gen_samples = np.array(samples).round(decimals=2)
    gen_labels = np.array(gen_labels)
    print(gen_samples.shape)
    print(gen_labels.shape)
    
    return gen_samples, gen_labels

# MLP model

In [14]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten
from tensorflow.keras.models import Sequential


def build_model(input_shape=(12,), num_classes=2):
    """

    :param input_shape: shape of input_data
    :param num_classes: number of classes
    :return: keras.model.sequential compiled with categorical cross-entropy loss
    """
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation="relu"),
        BatchNormalization(),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Flatten(),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.summary()
    return model

# Baseline Experiment

In [15]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

baseline_list =[]
for i in range(10):

    baseline_model = build_model()
    batch_size=32
    epochs=2
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
    history_baseline = baseline_model.fit(X_train, y_train_oh, batch_size=batch_size, 
                        epochs=epochs, validation_data=(X_test, test_y))
    score_baseline = baseline_model.evaluate(X_test, test_y, verbose=0)
    print('baseline test loss: ',score_baseline[0])
    print('baseline test accuracy: ', score_baseline[1] )

    y_pred_baseline_oh = baseline_model.predict(X_test)
    y_pred_baseline = y_pred_baseline_oh.argmax(axis=-1)
    baseline_list.append(classification_report(y_test, y_pred_baseline, output_dict=True))

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                416       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
__________________________________________________

2021-10-18 12:09:37.553883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 12:09:37.562607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-10-18 12:09:37.563376: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-10-18 12:09:37.564131: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library

31616/32561 [============================>.] - ETA: 0s - loss: 0.5161 - acc: 0.7722

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


32561/32561 [==============================] - 2s 53us/sample - loss: 0.5131 - acc: 0.7733 - val_loss: 0.3920 - val_acc: 0.8151
Epoch 2/2
32561/32561 [==============================] - 2s 46us/sample - loss: 0.3914 - acc: 0.8222 - val_loss: 0.3562 - val_acc: 0.8353
baseline test loss:  0.3562006383107384
baseline test accuracy:  0.8352681
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                416       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
___________________________________________

...

32561/32561 [==============================] - 2s 68us/sample - loss: 0.5218 - acc: 0.7653 - val_loss: 0.3887 - val_acc: 0.8216
Epoch 2/2
32561/32561 [==============================] - 2s 58us/sample - loss: 0.3993 - acc: 0.8177 - val_loss: 0.3635 - val_acc: 0.8347
baseline test loss:  0.3634840313625031
baseline test accuracy:  0.8347153
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_16 (Batc (None, 32)                128       
_________________________________________________________________
dense_25 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_17 (Batc (None, 64)                256       
___________________________________________

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


32561/32561 [==============================] - 2s 69us/sample - loss: 0.5068 - acc: 0.7759 - val_loss: 0.3731 - val_acc: 0.8296
Epoch 2/2
32561/32561 [==============================] - 2s 59us/sample - loss: 0.3949 - acc: 0.8171 - val_loss: 0.3604 - val_acc: 0.8312
baseline test loss:  0.3603959088060152
baseline test accuracy:  0.83115286
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_18 (Batc (None, 32)                128       
_________________________________________________________________
dense_28 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_19 (Batc (None, 64)                256       
__________________________________________

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


32561/32561 [==============================] - 2s 71us/sample - loss: 0.5020 - acc: 0.7750 - val_loss: 0.3693 - val_acc: 0.8292
Epoch 2/2
32561/32561 [==============================] - 2s 61us/sample - loss: 0.3917 - acc: 0.8170 - val_loss: 0.3469 - val_acc: 0.8423
baseline test loss:  0.3468981007453038
baseline test accuracy:  0.84227014


In [16]:
total_df = pd.DataFrame(baseline_list[0]).transpose()
print('number of runs: {}'.format(len(baseline_list)))
for r_dict in baseline_list[1:]:
    temp = pd.DataFrame(r_dict).transpose()
    total_df = total_df.add(temp)
    
average_10x = total_df/10.0
average_10x

number of runs: 10


,precision,recall,f1-score,support
0,0.861872,0.933164,0.895660,12435.000000
1,0.713396,0.514093,0.592436,3846.000000
accuracy,0.834169,0.834169,0.834169,0.834169
macro avg,0.787634,0.723629,0.744048,16281.000000
weighted avg,0.826798,0.834169,0.824031,16281.000000


# Augmentation Experiment

In [17]:
solver = tf.compat.v1.train.AdamOptimizer().minimize(vae_loss)
sess = tf.compat.v1.Session ()
sess.run(
tf.compat.v1.global_variables_initializer())

i = 0

for it in tqdm(range(50000)):
    ind = np.random.choice(X_train.shape[0], mb_size)
    X_mb = np.array(X_train[ind])
    y_mb = np.array(y_train_oh[ind])

    _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb, c: y_mb})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:40<00:00, 1249.63it/s]


In [ ]:
augment_list =[]
for i in range(10):
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
        
    gen_samples, gen_labels = generate_sample()
    x = np.concatenate([X_train, gen_samples])
    y = np.concatenate([y_train, gen_labels])
    x = np.array(x)
    y_oh = np.array(tf.keras.utils.to_categorical(y, num_classes=y_dim, dtype='float32'))
    
    aug_model = build_model()
    batch_size=32
    epochs=2
    hist = aug_model.fit(x, y_oh, batch_size=batch_size, epochs=epochs, 
                            validation_data=(X_test, test_y))
    y_pred_aug_oh = aug_model.predict(X_test)
    y_pred_aug = y_pred_aug_oh.argmax(axis=-1)
    augment_list.append(classification_report(y_test, y_pred_aug, output_dict=True) )

(6400, 12)
(6400,)
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_20 (Batc (None, 32)                128       
_________________________________________________________________
dense_31 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_21 (Batc (None, 64)                256       
_________________________________________________________________
flatten_10 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


38961/38961 [==============================] - 3s 70us/sample - loss: 0.4400 - acc: 0.8047 - val_loss: 0.4104 - val_acc: 0.8113
Epoch 2/2
38961/38961 [==============================] - 2s 61us/sample - loss: 0.3398 - acc: 0.8456 - val_loss: 0.3810 - val_acc: 0.8256
(6400, 12)
(6400,)
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_22 (Batc (None, 32)                128       
_________________________________________________________________
dense_34 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_23 (Batc (None, 64)                256       
_________________________________________________________________
flatten_11 (Flatten)         (No

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


38961/38961 [==============================] - 3s 79us/sample - loss: 0.4170 - acc: 0.8177 - val_loss: 0.3691 - val_acc: 0.8311
Epoch 2/2
38961/38961 [==============================] - 3s 78us/sample - loss: 0.3339 - acc: 0.8490 - val_loss: 0.3518 - val_acc: 0.8383
(6400, 12)
(6400,)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_24 (Batc (None, 32)                128       
_________________________________________________________________
dense_37 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_25 (Batc (None, 64)                256       
_________________________________________________________________
flatten_12 (Flatten)         (No

...

38961/38961 [==============================] - 4s 96us/sample - loss: 0.4117 - acc: 0.8185 - val_loss: 0.3767 - val_acc: 0.8308
Epoch 2/2
38961/38961 [==============================] - 3s 76us/sample - loss: 0.3354 - acc: 0.8463 - val_loss: 0.3560 - val_acc: 0.8366
(6400, 12)
(6400,)
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_36 (Batc (None, 32)                128       
_________________________________________________________________
dense_55 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_37 (Batc (None, 64)                256       
_________________________________________________________________
flatten_18 (Flatten)         (No

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


38961/38961 [==============================] - 4s 99us/sample - loss: 0.4273 - acc: 0.8151 - val_loss: 0.3796 - val_acc: 0.8278
Epoch 2/2
38961/38961 [==============================] - 3s 76us/sample - loss: 0.3354 - acc: 0.8484 - val_loss: 0.3557 - val_acc: 0.8373
(6400, 12)
(6400,)
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_38 (Batc (None, 32)                128       
_________________________________________________________________
dense_58 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_39 (Batc (None, 64)                256       
_________________________________________________________________
flatten_19 (Flatten)         (No

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


38961/38961 [==============================] - 4s 98us/sample - loss: 0.4334 - acc: 0.8115 - val_loss: 0.3933 - val_acc: 0.8266
Epoch 2/2
38400/38961 [============================>.] - ETA: 0s - loss: 0.3467 - acc: 0.8438

In [ ]:
import pandas as pd
total_df = pd.DataFrame(augment_list[0]).transpose()
print(len(augment_list))
for r_dict in augment_list[1:]:
    temp = pd.DataFrame(r_dict).transpose()
    total_df = total_df.add(temp)
    
average_10x = total_df/10.0
average_10x